In [27]:
# 1. MSA coefficients by land use type and intensity (from True Price table)
msa_value_by_land_use = {
    "natural_forest_minimal": 0.85,
    "natural_forest_light": 0.70,
    "natural_forest_intense": 0.50,
    "plantation_forest_minimal": 0.30,
    "plantation_forest_light": 0.25,
    "plantation_forest_intense": 0.20,
    "cropland_minimal": 0.40,
    "cropland_light": 0.30,
    "cropland_intense": 0.10,
    "pasture_light": 0.80,
    "pasture_moderate": 0.60,
    "pasture_heavy": 0.30,
    "urban": 0.05
}


# 2. Biome ESS values (€ per MSA·ha·year)
biome_ess_values = {
    'grassland': 2427,
    'temperate_forest': 1014,
    'wetland_inland': 14871,
    'wetland_coastal': 10939,
    'woodlands': 1369
}

# 3. Biome shares for the Netherlands
biome_share = {
    'grassland': 0.68,
    'temperate_forest': 0.14,
    'wetland_inland': 0.08,
    'wetland_coastal': 0.4,
    'woodlands': 0.6
}

# 4. Restoration costs (€ per MSA·ha), from True Price Table 4, adjusted for inflation to 2024
restoration_costs = {
    'grassland': 336.97,
    'temperate_forest': 4470.15,
    'wetland_inland': 19807.92,
    'wetland_coastal': 7902.03,
    'woodlands': 1702.26
}


In [30]:
# 5. Function to compute biodiversity impacts
def compute_land_use_metrics(use, biome_type, land_use_type, intensity, years_since_conversion):
    key = f"{land_use_type}_{intensity}"
    msa = msa_value_by_land_use.get(key, None)
    biome_fraction = biome_share.get(biome_type, None)
    ess_value = biome_ess_values.get(biome_type, 0)
    restoration_value = restoration_costs.get(biome_type, 0)

    if msa is None:
        raise ValueError(f"Invalid land use type or intensity: {key}")
    if biome_fraction is None:
        raise ValueError(f"Invalid biome type: {biome_type}")

    # Use (1 - MSA) to reflect biodiversity loss
    land_ob = use * biome_fraction * (1 - msa)
    land_cb = land_ob / years_since_conversion if years_since_conversion > 0 else None
    land_use_cost = round(land_ob * ess_value, 2)
    land_use_change_cost = round(land_cb * restoration_value, 2) if land_cb is not None else None

    return {
        "MSA value (biodiversity retained)": msa,
        "Biome Share": biome_fraction,
        "LAND-Ob (MSA.ha.yr/unit)": round(land_ob, 6),
        "LAND-Cb (MSA.ha/unit)": round(land_cb, 6) if land_cb else "N/A",
        "Land Use Cost (€ per unit)": f"{land_use_cost:.2f}",
        "Land Use Change Cost (€ per unit)": f"{land_use_change_cost:.2f}" if land_use_change_cost else "N/A"
    }


# 6. Compare food forest vs conventional agriculture EXAMPLE
food_forest = compute_land_use_metrics(
    use=0.002,                      # land use per unit of product (ha * yr / unit)
    biome_type='grassland',
    land_use_type='cropland',
    intensity='minimal',            # minimal interference
    years_since_conversion=20
)

conventional_agriculture = compute_land_use_metrics(
    use=0.002,
    biome_type='grassland',
    land_use_type='cropland',
    intensity='intense',
    years_since_conversion=20
)

food_forest, conventional_agriculture



({'MSA value (biodiversity retained)': 0.4,
  'Biome Share': 0.68,
  'LAND-Ob (MSA.ha.yr/unit)': 0.000816,
  'LAND-Cb (MSA.ha/unit)': 4.1e-05,
  'Land Use Cost (€ per unit)': '1.98',
  'Land Use Change Cost (€ per unit)': '0.01'},
 {'MSA value (biodiversity retained)': 0.1,
  'Biome Share': 0.68,
  'LAND-Ob (MSA.ha.yr/unit)': 0.001224,
  'LAND-Cb (MSA.ha/unit)': 6.1e-05,
  'Land Use Cost (€ per unit)': '2.97',
  'Land Use Change Cost (€ per unit)': '0.02'})